In [1]:
# --- Cell 1: Config, Registry (BASE_DIR / SUBJECTS / FEATURES / TITLE_MAP) ---
from __future__ import annotations

import os, json
from dataclasses import dataclass, asdict, field

# ==== Project registry（元ノートの値を踏襲） ====

BASE_DIR = r"""C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ\研究\実験結果"""

SUBJECTS = [
    ('0521', '因幡先生'),
    ('06021', '今村さん'),
    ('06022', '梅野さん'),
    ('06271', ''),
    ('06272', ''),
    ('06273', ''),
    ('06274', ''),
    ('06275', ''),
]

# FEATURES は「読み込み側の列名（キー）」で統一
FEATURES = [
    # Watch / Pulse / RR
    'watch_Heart_Rate(bpm)_mean',
    'watch_Sweat_Rate(mg/cm^2/min)_mean',
    'watch_Skin_Temperature(C)_mean',
    'HeartRate_BPM_mean',
    'RR_interval_sec_mean',

    # Face Temp（*_mean をキーに）
    'Face_Temp_Max_mean',
    'Face_Temp_Mean_mean',
    'Face_Temp_Max_Diff_mean',
    'Face_Temp_Mean_Diff_mean',

    # HRV / Nonlinear
    'RMSSD', 'SDSD', 'pNN50',
    'HF_power', 'LF_power', 'LF_HF_ratio',
    'SD1', 'SD2', 'CSI', 'CVI',
]

# 表示名（グラフタイトル・ラベル）への変換表
TITLE_MAP = {
    'Face_Temp_Max_mean': 'Face_Temp Max',
    'Face_Temp_Mean_mean': 'Face_Temp Mean',
    'Face_Temp_Max_Diff_mean': 'Face_Temp Max Diff',
    'Face_Temp_Mean_Diff_mean': 'Face_Temp Mean Diff',
    'HeartRate_BPM_mean': 'Pulse HR',
    'RR_interval_sec_mean': 'RR Interval',
    'RMSSD': 'RMSSD',
    'watch_Heart_Rate(bpm)_mean': 'Watch HR',
    'watch_Sweat_Rate(mg/cm^2/min)_mean': 'Watch ACC',
    'watch_Skin_Temperature(C)_mean': 'Watch Temp',
    'SDSD': 'SDSD',
    'pNN50': 'pNN50',
    'HF_power': 'HF Power',
    'LF_power': 'LF Power',
    'LF_HF_ratio': 'HF/LF',
    'CSI': 'CSI',
    'CVI': 'CVI',
    'SD1': 'SD1',
    'SD2': 'SD2',
}

def display_name(col: str) -> str:
    """グラフ表示用の名前（未登録はそのまま返す）"""
    return TITLE_MAP.get(col, col)

# ==== 派生コンフィグ（後続セルで共通利用） ====

@dataclass
class PlotStyle:
    linewidth: float = 1.5       # 線幅
    title_size: int = 30         # タイトル
    label_size: int = 24         # 軸ラベル
    legend_size: int = 20        # 凡例
    tick_size: int = 20          # 目盛

@dataclass
class TimeWindow:
    baseline_start: int = 1500   # sec
    baseline_end: int = 2100     # sec

@dataclass
class Paths:
    base_dir: str
    out_root: str
    def ensure(self):
        os.makedirs(self.out_root, exist_ok=True)
        os.makedirs(os.path.join(self.out_root, "fms"), exist_ok=True)
        os.makedirs(os.path.join(self.out_root, "group_mean"), exist_ok=True)
        os.makedirs(os.path.join(self.out_root, "effect"), exist_ok=True)

@dataclass
class Config:
    paths: Paths
    style: PlotStyle = field(default_factory=PlotStyle)     # ← default_factory
    window: TimeWindow = field(default_factory=TimeWindow)  # ← default_factory
    fms_sick_threshold: int = 2    # FMS >= 2 → Sick
    xtick_step_sec: int = 60       # X軸は60秒刻み（描画時は mm:ss）
    dpi: int = 200
    tight_layout: bool = True
    show_plots: bool = False

CFG = Config(
    paths=Paths(
        base_dir=BASE_DIR,
        out_root=r"./plots_refactored"
    )
)
CFG.paths.ensure()

print("Config:", json.dumps(asdict(CFG), ensure_ascii=False, indent=2))
print("SUBJECTS:", SUBJECTS)
print("FEATURES ({}):".format(len(FEATURES)), FEATURES)


Config: {
  "paths": {
    "base_dir": "C:\\Users\\taiki\\OneDrive - Science Tokyo\\デスクトップ\\研究\\実験結果",
    "out_root": "./plots_refactored"
  },
  "style": {
    "linewidth": 1.5,
    "title_size": 30,
    "label_size": 24,
    "legend_size": 20,
    "tick_size": 20
  },
  "window": {
    "baseline_start": 1500,
    "baseline_end": 2100
  },
  "fms_sick_threshold": 2,
  "xtick_step_sec": 60,
  "dpi": 200,
  "tight_layout": true,
  "show_plots": false
}
SUBJECTS: [('0521', '因幡先生'), ('06021', '今村さん'), ('06022', '梅野さん'), ('06271', ''), ('06272', ''), ('06273', ''), ('06274', ''), ('06275', '')]
FEATURES (19): ['watch_Heart_Rate(bpm)_mean', 'watch_Sweat_Rate(mg/cm^2/min)_mean', 'watch_Skin_Temperature(C)_mean', 'HeartRate_BPM_mean', 'RR_interval_sec_mean', 'Face_Temp_Max_mean', 'Face_Temp_Mean_mean', 'Face_Temp_Max_Diff_mean', 'Face_Temp_Mean_Diff_mean', 'RMSSD', 'SDSD', 'pNN50', 'HF_power', 'LF_power', 'LF_HF_ratio', 'SD1', 'SD2', 'CSI', 'CVI']


In [2]:
# --- Cell 2: Utilities (time mm:ss, savefig, Matplotlib style) ---
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# 時刻（秒）→ mm:ss 表記
def sec_to_mmss(x: float, pos=None) -> str:
    if x is None or np.isnan(x):
        return ""
    m = int(x // 60)
    s = int(x % 60)
    return f"{m:02d}:{s:02d}"

time_formatter = FuncFormatter(sec_to_mmss)

# ファイル名サニタイズ
def sanitize_filename(s: str) -> str:
    s = re.sub(r"[^\w\-.]+", "_", str(s).strip())
    return re.sub(r"_+", "_", s).strip("_")

# 画像保存（ディレクトリ自動作成・拡張子補完）
def savefig_safe(fig: plt.Figure, out_dir: str, filename: str, dpi: int | None = None) -> str:
    os.makedirs(out_dir, exist_ok=True)
    path = os.path.join(out_dir, sanitize_filename(filename))
    if not path.lower().endswith(".png"):
        path += ".png"
    fig.savefig(path, dpi=dpi or CFG.dpi)
    return path

# Matplotlibスタイル（既定方針に統一）
plt.rcParams.update({
    "axes.titlesize": CFG.style.title_size,
    "axes.labelsize": CFG.style.label_size,
    "legend.fontsize": CFG.style.legend_size,
    "xtick.labelsize": CFG.style.tick_size,
    "ytick.labelsize": CFG.style.tick_size,
    "lines.linewidth": CFG.style.linewidth,
})


In [3]:
# --- Cell 3: I/O helpers (CSV読み込み) ---
import pandas as pd

def scores_path(subject_id: str, person_name: str) -> str:
    """スコアファイル（*_scores.csv）のパスを返す"""
    d = os.path.join(CFG.paths.base_dir, f"{subject_id}{person_name}", "epoch_summary")
    return os.path.join(d, f"{subject_id}_scores.csv")

def epoch_path(subject_id: str, person_name: str) -> str:
    """エポックファイル（*_all_epoch_summary.csv）のパスを返す"""
    d = os.path.join(CFG.paths.base_dir, f"{subject_id}{person_name}", "epoch_summary")
    return os.path.join(d, f"{subject_id}_all_epoch_summary.csv")

def read_scores(subject_id: str, person_name: str) -> pd.DataFrame | None:
    """スコアCSVをDataFrameで読み込み"""
    p = scores_path(subject_id, person_name)
    if not os.path.exists(p):
        return None
    return pd.read_csv(p)

def read_epoch(subject_id: str, person_name: str) -> pd.DataFrame | None:
    """エポックCSVをDataFrameで読み込み"""
    p = epoch_path(subject_id, person_name)
    if not os.path.exists(p):
        return None
    return pd.read_csv(p)


In [4]:
# --- Cell 4: Sick / Non-Sick classification (OR rule: MaxFMS>=2 OR SSQ_Nausea>=20) ---
import numpy as np
import pandas as pd

def max_fms(fms_series: pd.Series) -> float:
    """FMSシリーズの最大値（NaN安全）"""
    if fms_series is None or len(fms_series) == 0:
        return np.nan
    return float(np.nanmax(pd.to_numeric(fms_series, errors="coerce").values))

def get_ssq_nausea(df_epoch: pd.DataFrame | None = None,
                   df_scores: pd.DataFrame | None = None,
                   nausea_col_epoch: str = "SSQ_Nausea",
                   nausea_col_scores: str = "SSQ_Nausea") -> float:
    """
    SSQ_Nausea の最大値を返す。scores優先（なければepoch）。
    見つからない場合は NaN。
    """
    vals = []
    if df_scores is not None and nausea_col_scores in df_scores.columns:
        vals.append(pd.to_numeric(df_scores[nausea_col_scores], errors="coerce").values)
    if df_epoch is not None and nausea_col_epoch in df_epoch.columns:
        vals.append(pd.to_numeric(df_epoch[nausea_col_epoch], errors="coerce").values)
    if not vals:
        return np.nan
    arr = np.concatenate(vals) if len(vals) > 1 else vals[0]
    return float(np.nanmax(arr)) if arr.size else np.nan

def classify_subject(df_epoch: pd.DataFrame,
                     df_scores: pd.DataFrame | None = None,
                     fms_col: str = "FMS",
                     fms_thr: int | None = None,
                     nausea_thr: float = 20.0) -> tuple[str, float, float]:
    """
    ORルールで群判定：
      Sick ⇔ (MaxFMS >= fms_thr) または (SSQ_Nausea >= nausea_thr)
    戻り値: (group, max_fms_value, ssq_nausea_max)
    """
    th = fms_thr if fms_thr is not None else CFG.fms_sick_threshold

    # MaxFMS
    maxf = np.nan
    if fms_col in df_epoch.columns:
        maxf = max_fms(df_epoch[fms_col])

    # SSQ_Nausea 最大
    nausea_max = get_ssq_nausea(df_epoch=df_epoch, df_scores=df_scores)

    cond_fms = (not np.isnan(maxf)) and (maxf >= th)
    cond_nau = (not np.isnan(nausea_max)) and (nausea_max >= nausea_thr)
    group = "Sick" if (cond_fms or cond_nau) else "Non-Sick"
    return group, maxf, nausea_max

# 互換ヘルパ（既存コードが (group, maxf) を想定している場合用）
def classify_subject_from_epoch(df_epoch: pd.DataFrame,
                                df_scores: pd.DataFrame | None = None,
                                fms_col: str = "FMS") -> tuple[str, float]:
    group, maxf, _ = classify_subject(df_epoch, df_scores, fms_col=fms_col)
    return group, maxf


In [5]:
# --- Cell 5: Load subjects, check files, and summarize groups ---
from typing import List, Tuple

def load_all_subjects(subjects: List[Tuple[str, str]]):
    """
    SUBJECTS の各被験者について
      - *_all_epoch_summary.csv / *_scores.csv を読み込み
      - ORルール（MaxFMS>=2 or SSQ_Nausea>=20）で群判定
    を行い、以下を返す:
      ALL      : [(sid, name, df_epoch, df_scores), ...]  ※両方読めたもののみ
      MISSING  : [(sid, name, missing_which), ...]        ※足りないファイルの情報
      summary  : DataFrame（被験者ごとの group, MaxFMS, SSQ_Nausea_max, paths）
    """
    loaded = []
    missing = []
    rows = []

    for sid, name in subjects:
        p_epoch  = epoch_path(sid, name)
        p_scores = scores_path(sid, name)

        df_e = read_epoch(sid, name)
        df_s = read_scores(sid, name)

        miss = []
        if df_e is None:
            miss.append("epoch")
        if df_s is None:
            miss.append("scores")

        if miss:
            missing.append((sid, name, ",".join(miss)))
            # 片方でも読めない場合は summary だけ埋めてスキップ
            maxf = np.nan
            nausea_max = np.nan
            group = "Non-Sick"
            if df_e is not None:
                group, maxf, nausea_max = classify_subject(df_e, df_s, fms_col="FMS")
            rows.append({
                "subject_id": sid,
                "person_name": name,
                "group": group,
                "MaxFMS": maxf,
                "SSQ_Nausea_max": nausea_max,
                "epoch_csv": p_epoch,
                "scores_csv": p_scores,
                "status": f"missing:{','.join(miss)}"
            })
            continue

        # 両方読めたケース：群判定
        group, maxf, nausea_max = classify_subject(df_e, df_s, fms_col="FMS")
        loaded.append((sid, name, df_e, df_s))
        rows.append({
            "subject_id": sid,
            "person_name": name,
            "group": group,
            "MaxFMS": maxf,
            "SSQ_Nausea_max": nausea_max,
            "epoch_csv": p_epoch,
            "scores_csv": p_scores,
            "status": "ok"
        })

    summary = pd.DataFrame(rows, columns=[
        "subject_id", "person_name", "group", "MaxFMS", "SSQ_Nausea_max",
        "epoch_csv", "scores_csv", "status"
    ])

    # 簡易サマリ表示
    n_ok = (summary["status"] == "ok").sum()
    n_ng = (summary["status"] != "ok").sum()
    g_cnt = summary["group"].value_counts(dropna=False).to_dict()

    print(f"[LOAD] ok={n_ok}, missing={n_ng}, total={len(summary)}")
    print(f"[GROUP] counts: {g_cnt}")
    if n_ng:
        print("[MISSING] first few:", missing[:5])

    display(summary)  # Jupyter上で見やすく

    return loaded, missing, summary

# 実行
ALL, MISSING, SUBJECT_SUMMARY = load_all_subjects(SUBJECTS)


[LOAD] ok=8, missing=0, total=8
[GROUP] counts: {'Sick': 6, 'Non-Sick': 2}


,subject_id,person_name,group,MaxFMS,SSQ_Nausea_max,epoch_csv,scores_csv,status
0,0521,因幡先生,Sick,2.0,9.54,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok
1,06021,今村さん,Sick,2.0,28.62,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok
2,06022,梅野さん,Sick,2.0,19.08,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok
3,06271,,Sick,2.0,38.16,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok
4,06272,,Sick,3.0,28.62,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok
5,06273,,Sick,3.0,47.70,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok
6,06274,,Non-Sick,1.0,9.54,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok
7,06275,,Non-Sick,0.0,0.00,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ...,ok


In [6]:
# --- Cell 6: Build FMS matrices using only Epoch_end ---
import numpy as np
import pandas as pd

def _pad_to_maxlen(arrs: list[np.ndarray], max_len: int) -> np.ndarray:
    """1次元配列のリストを max_len に右パディング（NaN）。"""
    if not arrs:
        return np.empty((0, max_len)) * np.nan
    out = np.full((len(arrs), max_len), np.nan, dtype=float)
    for i, a in enumerate(arrs):
        L = min(len(a), max_len)
        out[i, :L] = a[:L]
    return out

def build_group_fms_matrices_epoch_end(loaded, fms_col: str = "FMS"):
    """
    入力: loaded = [(sid, name, df_epoch, df_scores), ...]
    出力: (ref_time, sick_mat, nonsick_mat)
      - ref_time: 最長系列の Epoch_end（秒）
      - sick_mat / nonsick_mat: [被験者, 時間] の NaN パディング行列
    備考: 群判定はセル4の ORルール（MaxFMS>=2 or SSQ_Nausea>=20）
    """
    sick_list, nonsick_list, time_arrays = [], [], []

    for sid, name, df_epoch, df_scores in loaded:
        if "Epoch_end" not in df_epoch.columns or fms_col not in df_epoch.columns:
            print(f"[FMS][SKIP] {sid}{name}: requires columns ['Epoch_end','{fms_col}']")
            continue

        # 数値化 & 時系列ソート
        t = pd.to_numeric(df_epoch["Epoch_end"], errors="coerce").astype(float).values
        fms = pd.to_numeric(df_epoch[fms_col], errors="coerce").astype(float).values
        order = np.argsort(t)
        t = t[order]
        fms = fms[order]

        # 群判定（ORルール）
        group, _, _ = classify_subject(df_epoch, df_scores, fms_col=fms_col)

        time_arrays.append(t)
        if group == "Sick":
            sick_list.append(fms)
        else:
            nonsick_list.append(fms)

    if not time_arrays:
        print("[FMS] No valid time arrays found (need 'Epoch_end' and FMS).")
        return np.array([]), np.empty((0, 0)), np.empty((0, 0))

    # 参照時間軸は「最長のシリーズ」
    lengths = [len(t) for t in time_arrays]
    idx_long = int(np.argmax(lengths))
    ref_time = time_arrays[idx_long]
    max_len = len(ref_time)

    sick_mat    = _pad_to_maxlen(sick_list, max_len)
    nonsick_mat = _pad_to_maxlen(nonsick_list, max_len)

    tmin = float(np.nanmin(ref_time)) if len(ref_time) else np.nan
    tmax = float(np.nanmax(ref_time)) if len(ref_time) else np.nan
    print(f"[FMS] time_len={max_len}, sickN={sick_mat.shape[0]}, nonN={nonsick_mat.shape[0]}, "
          f"time_range=[{tmin:.0f},{tmax:.0f}], used_time_col=Epoch_end")
    return ref_time, sick_mat, nonsick_mat

# 実行
TIME_AXIS, FMS_SICK, FMS_NON = build_group_fms_matrices_epoch_end(ALL, fms_col="FMS")


[FMS] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], used_time_col=Epoch_end


In [7]:
# --- Cell 7: Plot FMS group mean ± SE (compact width++) ---
import numpy as np
import matplotlib.pyplot as plt

def mean_se(arr2d: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    m = np.nanmean(arr2d, axis=0)
    n = np.sum(~np.isnan(arr2d), axis=0)
    se = np.nanstd(arr2d, axis=0, ddof=1) / np.sqrt(np.maximum(n, 1))
    se[np.isinf(se)] = np.nan
    return m, se

def plot_fms_group_mean(time_sec: np.ndarray, sick_mat: np.ndarray, nonsick_mat: np.ndarray) -> str:
    if time_sec.size == 0:
        raise ValueError("Empty time axis.")

    # 横をさらに短く
    fig, ax = plt.subplots(figsize=(6.0, 4.6))
    fig.patch.set_facecolor("white")
    ax.set_facecolor("white")

    # データ描画（Sick=赤, NonSick=青）
    if sick_mat.size:
        m, se = mean_se(sick_mat)
        ax.plot(time_sec, m, label="Sick", color="red")
        ax.fill_between(time_sec, m - se, m + se, alpha=0.2, color="red")
    if nonsick_mat.size:
        m, se = mean_se(nonsick_mat)
        ax.plot(time_sec, m, label="Non-Sick", color="blue")
        ax.fill_between(time_sec, m - se, m + se, alpha=0.2, color="blue")

    # 軸設定
    ax.set_ylim(0, 4)
    ax.set_yticks(np.arange(0, 5, 1))
    ax.set_ylabel("FMS")

    # X軸：25分(=1500sec)を引いた 0～10 の整数表示
    offset_start = 1500
    ticks_pos = offset_start + 60 * np.arange(0, 11, 1)  # 0..10
    ax.set_xticks(ticks_pos)
    ax.set_xlim(offset_start, offset_start + 60 * 10)
    ax.set_xticklabels([str(i) for i in range(0, 11)])
    ax.set_xlabel("Time")

    # 指定時刻の縦点線（3:20=200sec, 6:40=400sec）
    ax.axvline(offset_start + 200, color="black", linestyle="--", linewidth=1.0, alpha=0.7)
    ax.axvline(offset_start + 400, color="black", linestyle="--", linewidth=1.0, alpha=0.7)

    # グリッド（x, y）
    ax.grid(True, which="major", axis="both", alpha=0.3)

    ax.legend(loc="best")
    ax.set_title("FMS group mean ± SE")

    if CFG.tight_layout:
        plt.tight_layout()

    out = savefig_safe(fig, os.path.join(CFG.paths.out_root, "fms"), "FMS_group_mean")
    if CFG.show_plots:
        plt.show()
    plt.close(fig)
    print(f"[SAVE] {out}")
    return out

# 実行
_ = plot_fms_group_mean(TIME_AXIS, FMS_SICK, FMS_NON)


[SAVE] ./plots_refactored\fms\FMS_group_mean.png


In [8]:
# --- Cell 8: Build feature Δ matrices (Epoch_end) with configurable baseline ---
import numpy as np
import pandas as pd
from typing import Tuple, Optional

# Cell 8 の compute_baseline_delta_epoch_end を次のように差し替え

def compute_baseline_delta_epoch_end(df_epoch, feature_col, baseline_start, baseline_end):
    if "Epoch_end" not in df_epoch.columns or feature_col not in df_epoch.columns:
        return None

    t = pd.to_numeric(df_epoch["Epoch_end"], errors="coerce").astype(float).values
    x = pd.to_numeric(df_epoch[feature_col], errors="coerce").astype(float).values

    mask = (t >= baseline_start) & (t <= baseline_end)
    if not np.any(mask):
        return pd.Series(np.full_like(x, np.nan, dtype=float), index=df_epoch.index, name=feature_col)

    x_base = x[mask]
    # ← 追加：ベースライン区間が全NaNなら安全にスキップ
    if not np.any(~np.isnan(x_base)):
        return pd.Series(np.full_like(x, np.nan, dtype=float), index=df_epoch.index, name=feature_col)

    baseline = float(np.nanmean(x_base))
    delta = x - baseline
    return pd.Series(delta, index=df_epoch.index, name=feature_col)


def _pad_to_maxlen(arrs: list[np.ndarray], max_len: int) -> np.ndarray:
    if not arrs:
        return np.empty((0, max_len)) * np.nan
    out = np.full((len(arrs), max_len), np.nan, dtype=float)
    for i, a in enumerate(arrs):
        L = min(len(a), max_len)
        out[i, :L] = a[:L]
    return out

def build_group_feature_delta_matrix_epoch_end(
    loaded,
    feature_col: str,
    baseline: Optional[Tuple[float, float]] = None,
):
    """
    入力: loaded = [(sid, name, df_epoch, df_scores), ...]
    出力: (ref_time, sick_mat, nonsick_mat)
      - ref_time: 最長の Epoch_end（秒）
      - sick_mat / nonsick_mat: Δ行列（NaNパディング）
    備考:
      - 群判定はセル4の ORルール（MaxFMS>=2 or SSQ_Nausea>=20）
      - baseline は (start, end)。未指定なら CFG.window を使用
    """
    if baseline is None:
        b_start, b_end = CFG.window.baseline_start, CFG.window.baseline_end
    else:
        b_start, b_end = baseline

    sick_list, nonsick_list, time_arrays = [], [], []

    for sid, name, df_epoch, df_scores in loaded:
        if "Epoch_end" not in df_epoch.columns or feature_col not in df_epoch.columns:
            continue

        # Δ作成（指定ベースライン）
        delta = compute_baseline_delta_epoch_end(df_epoch, feature_col, b_start, b_end)
        if delta is None:
            continue

        # 時系列にソート（Epoch_endで）
        t = pd.to_numeric(df_epoch["Epoch_end"], errors="coerce").astype(float).values
        order = np.argsort(t)
        t_sorted = t[order]
        d_sorted = pd.to_numeric(delta, errors="coerce").astype(float).values[order]

        # 群判定（OR）
        group, _, _ = classify_subject(df_epoch, df_scores, fms_col="FMS")

        time_arrays.append(t_sorted)
        if group == "Sick":
            sick_list.append(d_sorted)
        else:
            nonsick_list.append(d_sorted)

    if not time_arrays:
        print(f"[Δ] No valid series for feature='{feature_col}'. Check 'Epoch_end' and column name.")
        return np.array([]), np.empty((0, 0)), np.empty((0, 0))

    # 参照時間軸＝最長
    lengths = [len(t) for t in time_arrays]
    idx_long = int(np.argmax(lengths))
    ref_time = time_arrays[idx_long]
    max_len = len(ref_time)

    sick_mat    = _pad_to_maxlen(sick_list, max_len)
    nonsick_mat = _pad_to_maxlen(nonsick_list, max_len)

    tmin = float(np.nanmin(ref_time)) if len(ref_time) else np.nan
    tmax = float(np.nanmax(ref_time)) if len(ref_time) else np.nan
    print(f"[Δ:{feature_col}] time_len={max_len}, sickN={sick_mat.shape[0]}, nonN={nonsick_mat.shape[0]}, "
          f"time_range=[{tmin:.0f},{tmax:.0f}], baseline=[{b_start},{b_end}] (Epoch_end)")

    return ref_time, sick_mat, nonsick_mat

# 例：ベースライン 1500〜1530 を指定して最初のFEATUREで試す
if FEATURES:
    BASELINE_WINDOW = (1500, 1530)
    _t, _s, _n = build_group_feature_delta_matrix_epoch_end(ALL, FEATURES[0], baseline=BASELINE_WINDOW)


[Δ:watch_Heart_Rate(bpm)_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)


In [9]:
# --- Cell 9: Plot feature Δ (style matched to your sample) ---
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

def _d_and_se_per_time(sick_mat: np.ndarray, nonsick_mat: np.ndarray):
    """各時点で Cohen's d と SE(d)・p を計算（Welch t）。"""
    from scipy.stats import ttest_ind
    T = sick_mat.shape[1] if sick_mat.size else (nonsick_mat.shape[1] if nonsick_mat.size else 0)
    d_vals  = np.full(T, np.nan)
    se_vals = np.full(T, np.nan)
    p_vals  = np.full(T, np.nan)
    for t in range(T):
        xs = sick_mat[:, t]
        ys = nonsick_mat[:, t]
        xs = xs[~np.isnan(xs)]
        ys = ys[~np.isnan(ys)]
        n_s, n_n = len(xs), len(ys)
        if n_s >= 2 and n_n >= 2:
            # Welch t
            _, p = ttest_ind(xs, ys, equal_var=False)
            p_vals[t] = p
            # Cohen's d（pooled SD）
            mean_diff = np.mean(xs) - np.mean(ys)
            var_s = np.var(xs, ddof=1)
            var_n = np.var(ys, ddof=1)
            pooled = np.sqrt(((n_s - 1) * var_s + (n_n - 1) * var_n) / (n_s + n_n - 2))
            d = mean_diff / pooled if pooled > 0 else np.nan
            d_vals[t] = d
            # SE(d)（Hedges & Olkin 1985に基づく近似式）
            se_d = np.sqrt((n_s + n_n) / (n_s * n_n) + (d ** 2) / (2 * (n_s + n_n)))
            se_vals[t] = se_d
    return d_vals, se_vals, p_vals

def mean_se(arr2d: np.ndarray):
    m = np.nanmean(arr2d, axis=0)
    n = np.sum(~np.isnan(arr2d), axis=0)
    se = np.nanstd(arr2d, axis=0, ddof=1) / np.sqrt(np.maximum(n, 1))
    se[np.isinf(se)] = np.nan
    return m, se

def plot_feature_group_and_effect(feature_col: str,
                                  time_sec: np.ndarray,
                                  sick_mat: np.ndarray,
                                  nonsick_mat: np.ndarray) -> str:
    """体裁は提示コードに合わせ、ロジックは本リファクタ版を使用。"""
    if time_sec.size == 0:
        raise ValueError("Empty time axis.")
    disp = display_name(feature_col)

    # === X 軸（25:00=1500 を起点に 0..10 分ラベル／30秒刻み目盛） ===
    xticks = np.arange(1500, 2101, 30)  # 30s grid
    xticklabels = [f"{(t - 1500)//60}" if (t - 1500) % 60 == 0 else "" for t in xticks]
    vlines = np.linspace(1500, 2100, 4)[1:-1]  # 1700, 1900

    # === d, SE(d), p を計算 ===
    d_vals, se_vals, p_vals = _d_and_se_per_time(sick_mat, nonsick_mat)

    # === 図作成（サンプル体裁） ===
    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=True)
    fig.suptitle(f"{disp}", fontsize=30)
    colors = {"Sick": "red", "Non-Sick": "blue"}

    # ---- 左：Δ（群平均±SEM） ----
    ax1 = axes[0]
    # ベースライン帯（1500–1530 を既定。BASELINE_WINDOW があれば使用）
    b0, b1 = (1500, 1530)
    if "BASELINE_WINDOW" in globals() and isinstance(BASELINE_WINDOW, (tuple, list)) and len(BASELINE_WINDOW) == 2:
        b0, b1 = BASELINE_WINDOW
    ax1.axvspan(b0, b1, color="gray", alpha=0.2)

    # Sick
    if sick_mat.size:
        m, se = mean_se(sick_mat)
        ax1.plot(time_sec, m, label="Sick", linewidth=1.5, color=colors["Sick"])
        ax1.fill_between(time_sec, m - se, m + se, alpha=0.3, color=colors["Sick"])
    # Non-Sick
    if nonsick_mat.size:
        m, se = mean_se(nonsick_mat)
        ax1.plot(time_sec, m, label="Non-Sick", linewidth=1.5, color=colors["Non-Sick"])
        ax1.fill_between(time_sec, m - se, m + se, alpha=0.3, color=colors["Non-Sick"])

    # 軸体裁（サンプルに合わせる）
    ax1.set_xticks(xticks)
    ax1.set_xticklabels(xticklabels, fontsize=24)
    ax1.set_xlabel("Time)", fontsize=24)
    ax1.set_ylabel("change", fontsize=24)
    ax1.tick_params(labelsize=20)

    # Y軸指数表記（ScalarFormatter）
    fmt = ScalarFormatter(useMathText=True)
    fmt.set_powerlimits((-3, 4))
    fmt.set_scientific(True)
    fmt.set_useOffset(False)
    ax1.yaxis.set_major_formatter(fmt)

    # 縦点線 & グリッド
    for v in vlines:
        ax1.axvline(v, color="black", linestyle="--", linewidth=1)
    ax1.grid(True)
    ax1.set_xlim(1500, 2100)

    # 凡例（Sick → Non-Sick の順）
    handles, labels = ax1.get_legend_handles_labels()
    order = ["Sick", "Non-Sick"]
    pairs = sorted(zip(handles, labels), key=lambda hl: order.index(hl[1]) if hl[1] in order else 99)
    h_sorted, l_sorted = zip(*pairs) if pairs else ([], [])
    ax1.legend(h_sorted, l_sorted, fontsize=20)

    # ---- 右：効果量 d ＋ CI（95%）／点色は p-value（0–0.2、coolwarm_r） ----
    ax2 = axes[1]
    # 誤差棒（95%CI）— 点は描かず
    yerr = np.array([1.96 * s if np.isfinite(s) else 0.0 for s in se_vals])
    ax2.errorbar(time_sec, d_vals, yerr=yerr, fmt="none", ecolor="gray", elinewidth=1.5, capsize=4)

    # p による色
    norm = Normalize(vmin=0.0, vmax=0.2)
    cmap = plt.get_cmap("coolwarm_r")
    colors_p = [cmap(norm(p)) if np.isfinite(p) else (0.7, 0.7, 0.7, 0.3) for p in p_vals]
    ax2.scatter(time_sec, d_vals, c=colors_p, s=100, edgecolors="black", linewidths=0.5)

    # 折れ線（有効点のみ接続）
    valid = [(t, d) for t, d in zip(time_sec, d_vals) if np.isfinite(d)]
    if len(valid) > 1:
        t_ok, d_ok = zip(*valid)
        ax2.plot(t_ok, d_ok, color="black", linewidth=1.5, alpha=0.5)

    # カラーバー（0–0.2 固定）
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(np.array([p for p in p_vals if np.isfinite(p)]))
    cbar = plt.colorbar(sm, ax=ax2)
    cbar.set_label("p-value", fontsize=24)
    cbar.ax.tick_params(labelsize=20)

    # 軸体裁
    ax2.set_xticks(xticks)
    ax2.set_xticklabels(xticklabels, fontsize=20)
    ax2.set_xlabel("Time", fontsize=24)
    ax2.set_ylabel("Cohen's d", fontsize=24)
    ax2.set_ylim(-3, 3)
    yticks = np.linspace(-3, 3, 7)
    ax2.set_yticks(yticks)
    ax2.set_yticklabels([f"{y:.2f}" for y in yticks], fontsize=24)
    ax2.grid(True)

    # ベースライン帯／縦点線
    ax2.axvspan(b0, b1, color="gray", alpha=0.2)
    for v in vlines:
        ax2.axvline(v, color="black", linestyle="--", linewidth=1)

    # 仕上げ
    plt.tight_layout()
    out = savefig_safe(fig, os.path.join(CFG.paths.out_root, "group_mean"), f"{disp}_group_and_effect")
    if CFG.show_plots:
        plt.show()
    plt.close(fig)
    print(f"[SAVE] {out}")
    return out


In [10]:
# --- Cell 10: Run all FEATURES (baseline = 1500–1530) ---
BASELINE_WINDOW = (1500, 1530)

generated = []
skipped = []

for col in FEATURES:
    # Δ行列を作成（Epoch_end基準、ORルール）
    t, s_mat, n_mat = build_group_feature_delta_matrix_epoch_end(
        ALL, feature_col=col, baseline=BASELINE_WINDOW
    )
    if t.size == 0:
        print(f"[SKIP] {col}: no valid series (missing 'Epoch_end' or column not found)")
        skipped.append(col)
        continue

    try:
        out = plot_feature_group_and_effect(col, t, s_mat, n_mat)
        print(f"[OK]   {display_name(col)} -> {out}")
        generated.append((col, out))
    except Exception as e:
        print(f"[SKIP] {col}: plot failed ({e})")
        skipped.append(col)

print("\nSummary:")
print(f"  Generated: {len(generated)}")
print(f"  Skipped  : {len(skipped)}")
if skipped:
    print("  Skipped list:", skipped)


[Δ:watch_Heart_Rate(bpm)_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\Watch_HR_group_and_effect.png
[OK]   Watch HR -> ./plots_refactored\group_mean\Watch_HR_group_and_effect.png
[Δ:watch_Sweat_Rate(mg/cm^2/min)_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\Watch_ACC_group_and_effect.png
[OK]   Watch ACC -> ./plots_refactored\group_mean\Watch_ACC_group_and_effect.png
[Δ:watch_Skin_Temperature(C)_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\Watch_Temp_group_and_effect.png
[OK]   Watch Temp -> ./plots_refactored\group_mean\Watch_Temp_group_and_effect.png
[Δ:HeartRate_BPM_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\Pulse_HR_group_and_effect.png
[OK]   Pul

C:\Users\taiki\AppData\Local\Temp\ipykernel_21320\3051130016.py:38: RuntimeWarning: Mean of empty slice
  m = np.nanmean(arr2d, axis=0)
c:\Users\taiki\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[SAVE] ./plots_refactored\group_mean\pNN50_group_and_effect.png
[OK]   pNN50 -> ./plots_refactored\group_mean\pNN50_group_and_effect.png
[Δ:HF_power] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)


C:\Users\taiki\AppData\Local\Temp\ipykernel_21320\3051130016.py:38: RuntimeWarning: Mean of empty slice
  m = np.nanmean(arr2d, axis=0)
c:\Users\taiki\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[SAVE] ./plots_refactored\group_mean\HF_Power_group_and_effect.png
[OK]   HF Power -> ./plots_refactored\group_mean\HF_Power_group_and_effect.png
[Δ:LF_power] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)


C:\Users\taiki\AppData\Local\Temp\ipykernel_21320\3051130016.py:38: RuntimeWarning: Mean of empty slice
  m = np.nanmean(arr2d, axis=0)
c:\Users\taiki\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[SAVE] ./plots_refactored\group_mean\LF_Power_group_and_effect.png
[OK]   LF Power -> ./plots_refactored\group_mean\LF_Power_group_and_effect.png
[Δ:LF_HF_ratio] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)


C:\Users\taiki\AppData\Local\Temp\ipykernel_21320\3051130016.py:38: RuntimeWarning: Mean of empty slice
  m = np.nanmean(arr2d, axis=0)
c:\Users\taiki\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


[SAVE] ./plots_refactored\group_mean\HF_LF_group_and_effect.png
[OK]   HF/LF -> ./plots_refactored\group_mean\HF_LF_group_and_effect.png
[Δ:SD1] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\SD1_group_and_effect.png
[OK]   SD1 -> ./plots_refactored\group_mean\SD1_group_and_effect.png
[Δ:SD2] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\SD2_group_and_effect.png
[OK]   SD2 -> ./plots_refactored\group_mean\SD2_group_and_effect.png
[Δ:CSI] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\CSI_group_and_effect.png
[OK]   CSI -> ./plots_refactored\group_mean\CSI_group_and_effect.png
[Δ:CVI] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\CVI_group_and_effect.png
[OK]   CVI -> ./plots_refact

In [11]:
# --- Cell 11 (optional): Export d / SE(d) / p per time to CSV ---
import pandas as pd
import os
import numpy as np

os.makedirs(os.path.join(CFG.paths.out_root, "group_mean", "effects_csv"), exist_ok=True)

def export_effect_table(feature_col: str, baseline=(1500, 1530)) -> str | None:
    # Δ行列を再作成
    t, s_mat, n_mat = build_group_feature_delta_matrix_epoch_end(ALL, feature_col, baseline=baseline)
    if t.size == 0:
        print(f"[SKIP] {feature_col}: no data")
        return None

    # d, SE(d), p を取得（Cell9のヘルパーを利用）
    d_vals, se_vals, p_vals = _d_and_se_per_time(s_mat, n_mat)

    df = pd.DataFrame({
        "Time_sec": t.astype(float),
        "Time_min_from25": (t - 1500) / 60.0,
        "Cohen_d": d_vals,
        "SE_d": se_vals,
        "p_value": p_vals,
    })

    disp = display_name(feature_col)
    out_dir = os.path.join(CFG.paths.out_root, "group_mean", "effects_csv")
    out_path = os.path.join(out_dir, sanitize_filename(f"{disp}_effect_p.csv"))
    df.to_csv(out_path, index=False)
    print(f"[SAVE] {out_path}")
    return out_path

# まとめて実行
for col in FEATURES:
    export_effect_table(col, baseline=(1500, 1530))


[Δ:watch_Heart_Rate(bpm)_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\effects_csv\Watch_HR_effect_p.csv
[Δ:watch_Sweat_Rate(mg/cm^2/min)_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\effects_csv\Watch_ACC_effect_p.csv
[Δ:watch_Skin_Temperature(C)_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\effects_csv\Watch_Temp_effect_p.csv
[Δ:HeartRate_BPM_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\effects_csv\Pulse_HR_effect_p.csv
[Δ:RR_interval_sec_mean] time_len=20, sickN=6, nonN=2, time_range=[1530,2100], baseline=[1500,1530] (Epoch_end)
[SAVE] ./plots_refactored\group_mean\effects_csv\RR_Interval_effect_p.csv
[Δ:Face_Temp_Max_mean] time_len=20, sickN=6, nonN=2